In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import calendar
from sklearn.preprocessing import LabelEncoder
#忽略所有warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 加载所有数据
train = pd.read_csv('./store-sales-time-series-forecasting/train.csv',  parse_dates=['date'])
test = pd.read_csv('./store-sales-time-series-forecasting/test.csv',  parse_dates=['date'])
stores = pd.read_csv('./store-sales-time-series-forecasting/stores.csv')
oil = pd.read_csv('./store-sales-time-series-forecasting/oil.csv',  parse_dates=['date'])
holidays = pd.read_csv('./store-sales-time-series-forecasting/holidays_events.csv',  parse_dates=['date'])

In [3]:
# 2. 转换日期格式（确保所有数据中日期字段为 datetime 格式）
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])
oil['date'] = pd.to_datetime(oil['date'])
holidays['date'] = pd.to_datetime(holidays['date'])

In [4]:
# 打印数据集大小
print("Train dataset size:", train.shape)

Train dataset size: (3000888, 6)


In [5]:
# 3. 处理 holidays_events 数据
# 根据说明：Transferred 类型视作正常日，其它假日类型认为是假日
holidays['is_holiday'] = holidays['type'].apply(lambda x: 0 if x == 'Transfer' else 1)
# 如果一天内有多个事件，则取最大值（即只要有非 Transfer 的事件，该天就视作假日）
holidays_daily = holidays.groupby('date')['is_holiday'].max().reset_index()


In [6]:
# 4. 合并外部数据
# 4.1 合并油价数据：按日期合并到训练集和测试集（左连接保证保留所有主数据的日期）
train = pd.merge(train, oil[['date', 'dcoilwtico']], on='date', how='left')
test = pd.merge(test, oil[['date', 'dcoilwtico']], on='date', how='left')


In [7]:
# 4.2 合并商店数据：按 store_nbr 合并
train = pd.merge(train, stores, on='store_nbr', how='left')
test = pd.merge(test, stores, on='store_nbr', how='left')

In [8]:
# 4.3 合并 holidays 数据：按日期合并
train = pd.merge(train, holidays_daily, on='date', how='left')
test = pd.merge(test, holidays_daily, on='date', how='left')
# 对于没有合并到的日期，填充 is_holiday = 0
train['is_holiday'] = train['is_holiday'].fillna(0)
test['is_holiday'] = test['is_holiday'].fillna(0)

In [9]:
# 5. 生成日期特征
def add_date_features(df):
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['weekday'] = df['date'].dt.weekday  # Monday=0, Sunday=6
    # 发薪日：每月15日和每月最后一天
    df['is_payday'] = df['date'].apply(lambda x: 1 if (x.day == 15 or x.day == calendar.monthrange(x.year, x.month)[1]) else 0)
    return df

In [10]:
train = add_date_features(train)
test = add_date_features(test)

In [11]:
# 6. 生成特殊事件特征：针对2016-04-16地震
# 例如：假设地震对4月16日至5月底的销售产生影响，这里创建一个标记
def add_earthquake_feature(df):
    # 根据需要调整时间窗口
    earthquake_start = pd.to_datetime('2016-04-16')
    earthquake_end = pd.to_datetime('2016-05-31')
    df['is_earthquake_period'] = df['date'].apply(lambda x: 1 if earthquake_start <= x <= earthquake_end else 0)
    return df

In [12]:
train = add_earthquake_feature(train)
test = add_earthquake_feature(test)

In [13]:
# 7. 检查缺失值情况（这里只是简单地打印各个数据集的缺失情况）
print("Train missing values:")
print(train.isnull().sum())
print("\nTest missing values:")
print(test.isnull().sum())

Train missing values:
id                           0
date                         0
store_nbr                    0
family                       0
sales                        0
onpromotion                  0
dcoilwtico              928422
city                         0
state                        0
type                         0
cluster                      0
is_holiday                   0
year                         0
month                        0
day                          0
weekday                      0
is_payday                    0
is_earthquake_period         0
dtype: int64

Test missing values:
id                         0
date                       0
store_nbr                  0
family                     0
onpromotion                0
dcoilwtico              7128
city                       0
state                      0
type                       0
cluster                    0
is_holiday                 0
year                       0
month                      0
day     

In [14]:
# 8. 检查油价相关的重复列（可选）
print("油价相关的前几行数据：")
print(train[['dcoilwtico']].head())

油价相关的前几行数据：
   dcoilwtico
0         NaN
1         NaN
2         NaN
3         NaN
4         NaN


In [15]:
# 9. 删除重复的油价列，只保留 dcoilwtico 这一列
# train = train.drop(['dcoilwtico_x', 'dcoilwtico_y'], axis=1)
# test = test.drop(['dcoilwtico_x', 'dcoilwtico_y'], axis=1)

In [16]:
# 10. 对 dcoilwtico 列进行缺失值填充
# 先用前向填充，再用后向填充（以防数据开头处仍有缺失）
train['dcoilwtico'] = train['dcoilwtico'].fillna(method='ffill').fillna(method='bfill')
test['dcoilwtico'] = test['dcoilwtico'].fillna(method='ffill').fillna(method='bfill')


In [17]:
# 11. 检查处理后的缺失值情况
print("处理后的Train缺失值情况：")
print(train.isnull().sum())
print("\n处理后的Test缺失值情况：")
print(test.isnull().sum())

处理后的Train缺失值情况：
id                      0
date                    0
store_nbr               0
family                  0
sales                   0
onpromotion             0
dcoilwtico              0
city                    0
state                   0
type                    0
cluster                 0
is_holiday              0
year                    0
month                   0
day                     0
weekday                 0
is_payday               0
is_earthquake_period    0
dtype: int64

处理后的Test缺失值情况：
id                      0
date                    0
store_nbr               0
family                  0
onpromotion             0
dcoilwtico              0
city                    0
state                   0
type                    0
cluster                 0
is_holiday              0
year                    0
month                   0
day                     0
weekday                 0
is_payday               0
is_earthquake_period    0
dtype: int64


In [18]:
# 对字符串特征进行标签编码
# 需要编码的列：family、city、state、type
# 注意：为保证训练和测试中编码一致，先在训练集上 fit，再 transform 测试集
for col in ['family', 'city', 'state', 'type']:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col].astype(str))
    test[col] = le.transform(test[col].astype(str))

In [21]:
# 查看处理结果
print("处理后的Train前几行：")
print(train.head())
print("Train dataset size:", train.shape)

处理后的Train前几行：
   id       date  store_nbr  family  sales  onpromotion  dcoilwtico  city  \
0   0 2013-01-01          1       0    0.0            0       93.14    18   
1   1 2013-01-01          1       1    0.0            0       93.14    18   
2   2 2013-01-01          1       2    0.0            0       93.14    18   
3   3 2013-01-01          1       3    0.0            0       93.14    18   
4   4 2013-01-01          1       4    0.0            0       93.14    18   

   state  type  cluster  is_holiday  year  month  day  weekday  is_payday  \
0     12     3       13         1.0  2013      1    1        1          0   
1     12     3       13         1.0  2013      1    1        1          0   
2     12     3       13         1.0  2013      1    1        1          0   
3     12     3       13         1.0  2013      1    1        1          0   
4     12     3       13         1.0  2013      1    1        1          0   

   is_earthquake_period  
0                     0  
1       

In [20]:
# 12. 保存预处理后的数据（根据需要进行保存）
train.to_csv('./favourite/train_preprocessed.csv', index=False)
test.to_csv('./favourite/test_preprocessed.csv', index=False)

print("数据预处理完成，预处理后的数据已保存。")

数据预处理完成，预处理后的数据已保存。


In [22]:
#将train_preprocessed.csv的前5行保存到train.csv中
train = pd.read_csv('./favourite/train_preprocessed.csv')
train.head().to_csv('./favourite/train.csv', index=False)